# Augmented Weighted Tchebycheff

    Compute one nondominated point by application of the Tchebycheff theory.
    Application to the bi-objective 01 unidimensionnal knapsack problem. 
        
    In  Steuer, R.E., Choo, EU. An interactive weighted Tchebycheff procedure 
        for multiple objective programming. Mathematical Programming 26, 326–344 (1983).    

## 1) An instance of 01 Unidimensional Knapsack Problem with two objectives

In [ ]:
# Profits:

    p = [ 13 10  3 16 12 11  1  9 19 13 ;     # profit 1
           1 10  3 13 12 19 16 13 11  9  ]    # profit 2

# Weights:

    w  = [ 4, 4, 3, 5, 5, 3, 2, 3, 5, 4  ] 

# Capacity:

    c  = 19

# Dimensions:

    d,n = size(p)   # number of objectives and number of variables 


## 2) A set of values for parameters $\lambda$, $r^*$, $\rho$

In [ ]:
# The weights:

    λ = [ 0.25 , 0.75 ]

# The reference point:

    rp = [ 100 , 100 ] 

# The augmented value:

    ρ = 0.001


## 3) The MILP model

##### Create a JuMP model using GLPK as MIP solver

In [ ]:
    using JuMP, GLPK
    model = Model(GLPK.Optimizer)

##### Defintions related to the 2-01UKP model

The $n$ binary variables

In [ ]:
    @variable(model, x[1:n], Bin)

The constraint

In [ ]:
    @constraint(model, sum(w[i] * x[i] for i in 1:n) ≤ c)

The $d$ objective functions

In [ ]:
    @expression(model, z[k=1:d], sum(p[k,j] * x[j] for j in 1:n))

##### Definitions related to the Tchebycheff model

The variable

In [ ]:
    @variable(model, α ≥ 0) 

The $d$ constraints

In [ ]:
    @constraint(model, con[k=1:d], α ≥  λ[k] * (rp[k] -  z[k]))

The objective to minimize

In [ ]:
    @objective(model, Min, α + ρ * sum((rp[k] -  z[k]) for k=1:d))

## 3) Optimize, query and display the results

In [ ]:
# optimize:

    set_silent(model)
    solve_time_sec = 0.0
    optimize!(model)
    @assert is_solved_and_feasible(model) "Error: optimal solution not found"
    solve_time_sec += solve_time(model)


In [ ]:
# query the results:

    f1_Opt = sum(p[1,j] * value(x[j]) for j in 1:n)
    f2_Opt = sum(p[2,j] * value(x[j]) for j in 1:n)
    x_Opt  = value.(x)

# display the results:

    println("zOpt = ", [round(Int, value(f1_Opt)), round(Int, value(f2_Opt))] )
    println("XOpt = ", [round(Int, x_Opt[i]) for i in 1:n])
    println("time = $(round(solve_time_sec, digits=3)) (s)")


-----

## Summary

In [2]:
using JuMP, GLPK

In [12]:
function  SolveAugmentedWeightedTchebycheff(  
            solver::DataType, 
            p::Matrix{Int64},  w::Vector{Int64},  c::Int64,  
            λ::Vector{Float64}, rp::Vector{Int64}, ρ::Float64
        )
           
    d,n = size(p)

    model = Model(solver)  

    @variable(model, x[1:n], Bin)                              
    @constraint(model, sum(w[i] * x[i] for i in 1:n) ≤ c)        
    @expression(model, z[k=1:d], sum(p[k,j] * x[j] for j in 1:n))   

    @variable(model, α ≥ 0)        
    @constraint(model, con[k=1:d], α ≥  λ[k] * (rp[k] -  z[k]))
    @objective(model, Min, α + ρ * sum((rp[k] -  z[k]) for k=1:d))
        
    set_silent(model)
    solve_time_sec = 0.0
    optimize!(model)
    @assert is_solved_and_feasible(model) "Error: optimal solution not found"
    solve_time_sec += round( solve_time(model) , digits=3)
        
    f1_Opt = round(Int, sum(p[1,j] * value(x[j]) for j in 1:n) )
    f2_Opt = round(Int, sum(p[2,j] * value(x[j]) for j in 1:n) )
    x_Opt  = [round(Int, value(x[i])) for i in 1:n]       

    return [f1_Opt, f2_Opt], x_Opt , solve_time_sec
end

SolveAugmentedWeightedTchebycheff (generic function with 1 method)

In [4]:
function generate_MO01UKP(n = 10, o = 2, max_ci = 100, max_wi = 30)

    p = rand(1:max_ci,o,n)      # c_i \in [1,max_ci]   # profits
    w = rand(1:max_wi,n)        # w_i \in [1,max_wi]   # weight
    c = round(Int64, sum(w)/2)                         # capacity
                
    return p, w, c
end

generate_MO01UKP (generic function with 5 methods)

In [13]:
solver = GLPK.Optimizer

p,w,c = generate_MO01UKP()

λ  = [0.25,0.75]
rp = [80,80]
ρ  = 0.001

z,x,t = SolveAugmentedWeightedTchebycheff( solver,  p, w, c,  λ, rp, ρ)

println("zOpt = ", z)
println("XOpt = ", x)
println("time = $t (s)") 

zOpt = [325, 434]
XOpt = [1, 1, 1, 0, 1, 1, 0, 1, 0, 1]
time = 0.0 (s)
